In [1]:
import pandas as pd
import os 
from pathlib import Path
import plotly.express as px

In [2]:
df = pd.read_excel("output/output.xlsx")
df.head()

,Unnamed: 0,avs_id,model,seed,timestamp,text,flesch_kincaid_grade,flesch_reading_ease,smog_index,coleman_liau_index,...,diff_difficult_words,diff_linsear_write_formula,diff_gunning_fog,diff_text_standard,diff_fernandez_huerta,diff_szigriszt_pazos,diff_gutierrez_polini,diff_crawford,diff_gulpease_index,diff_osman
0,0,0,unmodified,0,NaN,\nDepartment: ANESTHESIA PREOPERATIVE PROGRAM\...,10.8,51.89,13.3,14.10,...,0,0.0,0.0,Cannot compute,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,unmodified,0,NaN,Department: ANESTHESIA PREOPERATIVE PROGRAM\nD...,10.2,53.61,13.1,13.92,...,0,0.0,0.0,Cannot compute,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,unmodified,0,NaN,\nDepartment: ANESTHESIA PREOPERATIVE PROGRAM\...,10.5,52.80,13.0,13.98,...,0,0.0,0.0,Cannot compute,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,unmodified,0,NaN,Department: ANESTHESIA PREOPERATIVE PROGRAM\nD...,9.9,54.22,12.7,11.89,...,0,0.0,0.0,Cannot compute,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,unmodified,0,NaN,Department: ANESTHESIA PREOPERATIVE PROGRAM\nD...,10.7,52.09,13.5,14.16,...,0,0.0,0.0,Cannot compute,0.0,0.0,0.0,0.0,0.0,0.0


accuracy analysis protocol

 | Domain | 	Is specific example item present?         | Complete (1) | 	Partial (0.5) | 	Absent (0) | Inaccurate (1/0)_ | 
 |----|--------------------------------------------|--------------|----------------|-------------|-------------------| 
|Logistics| 	Procedure date  |              |                |             |                   |					
| Eating and drinking	| Description includes examples of clear liquids |              |                |             |                   |	
| Transportation	| Request to have a driver |              |                |             |                   |	
|Personal items |	Disallow contact lenses		 |              |                |             |                   |	
|Medication management |	Instructions for NSAIDs (disallowed except in endo) |              |                |             |                   |		
						

In [4]:
unmodified_df = df.query("model == 'unmodified'")
unmodified_df

,Unnamed: 0,avs_id,model,seed,timestamp,text,flesch_kincaid_grade,flesch_reading_ease,smog_index,coleman_liau_index,...,diff_difficult_words,diff_linsear_write_formula,diff_gunning_fog,diff_text_standard,diff_fernandez_huerta,diff_szigriszt_pazos,diff_gutierrez_polini,diff_crawford,diff_gulpease_index,diff_osman
0,0,0,unmodified,0,NaN,\nDepartment: ANESTHESIA PREOPERATIVE PROGRAM\...,10.8,51.89,13.3,14.10,...,0,0.0,0.0,Cannot compute,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,unmodified,0,NaN,Department: ANESTHESIA PREOPERATIVE PROGRAM\nD...,10.2,53.61,13.1,13.92,...,0,0.0,0.0,Cannot compute,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,unmodified,0,NaN,\nDepartment: ANESTHESIA PREOPERATIVE PROGRAM\...,10.5,52.80,13.0,13.98,...,0,0.0,0.0,Cannot compute,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,unmodified,0,NaN,Department: ANESTHESIA PREOPERATIVE PROGRAM\nD...,9.9,54.22,12.7,11.89,...,0,0.0,0.0,Cannot compute,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,unmodified,0,NaN,Department: ANESTHESIA PREOPERATIVE PROGRAM\nD...,10.7,52.09,13.5,14.16,...,0,0.0,0.0,Cannot compute,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
import re

def find_sentences_with_words(text, words):
    # Splitting the text into sentences
    sentences = re.split(r'(?<=[.!?]) +|\n', text)

    # Preparing the list of words to search for
    words_to_find = '|'.join(map(re.escape, words))

    # Finding sentences that contain any of the words
    matching_sentences = [sentence for sentence in sentences if re.search(words_to_find, sentence, re.IGNORECASE)]
    
    if len(matching_sentences) == 1:
        return matching_sentences[0]
    if len(matching_sentences) >1:
        return_value = "More than one sentence found: " + "  ".join(matching_sentences)
        return return_value
    
    # print("No sentences found")
    return_value = "No sentences found"
    return return_value

# Example Usage
text = "Your large text goes here. It can be multiple sentences (like apple). This script will find specific words like apple, banana, or cherry."
words = ["apple", "banana", "cherry"]

found_sentences = find_sentences_with_words(text, words)
print(found_sentences)

More than one sentence found: It can be multiple sentences (like apple).  This script will find specific words like apple, banana, or cherry.


In [32]:
df["logistics_accuracy"] = 0
df["logistics_completion"] = 0
df["logistics_sentence"] = ""

df["eating_accuracy"] = 0
df["eating_completion"] = 0
df["eating_sentence"] = ""

df["transportation_accuracy"] = 0
df["transportation_completion"] = 0
df["transportation_sentence"] = ""

df["personal_items_accuracy"] = 0
df["personal_items_completion"] = 0
df["personal_items_sentence"] = ""

df["meds_accuracy"] = 0
df["meds_completion"] = 0
df["meds_sentence"] = ""

for index, row in df.iterrows():
    
    timestamp = row['timestamp']
    avs_text = row['text']

    # Logistics
    # Procedure date

    sentence = find_sentences_with_words(avs_text, ["1/14/24", "01/14/24", "14", "January", "2024"])
    if sentence == "No sentences found":
        print("Logistics:" + str(index) + " " + timestamp + " " + sentence)
    df.loc[index, 'logistics_sentence'] = sentence
    
    # Eating and drinking
    # Description includes examples of clear liquids
    sentence = find_sentences_with_words(avs_text, [ "tea", "water", "juice", "drinks"])
    if sentence == "No sentences found":
        print("Eating:" + str(index) + " " + timestamp + " " + sentence)
    df.loc[index, 'eating_sentence'] = sentence 
    
    # Transportation
    # Request to have a driver
    sentence = find_sentences_with_words(avs_text, [ "drive" ])
    if sentence == "No sentences found":
        print("Transport:" + str(index) + " " + timestamp + " " + sentence)
    df.loc[index, 'transportation_sentence'] = sentence 
    
    # Personal items
    # Disallow contact lenses
    sentence = find_sentences_with_words(avs_text, [ "contacts", "lenses", "lens" ])
    if sentence == "No sentences found":
        print("Personal items:" + str(index) + " " + timestamp + " " + sentence)
    df.loc[index, 'personal_items_sentence'] = sentence 
       
    
    # Medication management
    # Instructions for NSAIDs (disallowed except in endo)
    sentence = find_sentences_with_words(avs_text, [ "NSAIDS", "ibuprofen", "aspirin" ])
    if sentence == "No sentences found":
        print("Meds:" + str(index) + " " + timestamp + " " + sentence)
    df.loc[index, 'meds_sentence'] = sentence 
    

In [33]:
df.to_excel("output/output_with_accuracy_analysis.xlsx")
df.to_csv("output/output_with_accuracy_analysis.csv")
